In [39]:
import cv2
import pandas as pd
import numpy as np
import opencv_jupyter_ui as jcv2
import os
from pathlib import Path
from feat import Detector
from IPython.display import Image
from feat.utils import FEAT_EMOTION_COLUMNS

from torch import nn
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import MinMaxScaler

In [20]:
detector = Detector(device="cuda")

In [44]:
class MLP(nn.Module):
    def __init__(self, features_in=2, features_out=3):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(features_in, 15),
            nn.Linear(15, features_out)
            #nn.Linear(features_in, features_out)
        )
    def forward(self, input):
        return self.net(input)

In [45]:
path=Path(os.getcwd()).parent
DIR_PATH=str(path) + '\\'

device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load(Path( DIR_PATH + f'models/ValAr/model1.pt') ).to(device)

In [46]:
class DiffFER(Dataset):
    def __init__(self, data):
        super().__init__()
        sc = MinMaxScaler()
        # everything in pytorch needs to be a tensor
        label_cols=['valence', 'arousal']
        self.labels = torch.tensor(sc.fit_transform(data[label_cols].to_numpy(dtype=np.float32)))
        self.inputs = torch.tensor(data.drop(label_cols, axis=1).to_numpy(dtype=np.float32))

    def __getitem__(self, index):
        return self.inputs[index], self.labels[index]

    def __len__(self):
        return len(self.inputs)

In [47]:
dataset = pd.read_csv(DIR_PATH + r'data/extracted_df.csv')
cols=['AU06', 'AU07', 'AU12', 'AU14', 'AU43',  'valence', 'arousal']
data = dataset[cols]
dat=DiffFER(data)

In [48]:
def return_emo_state(arr):
    if arr[0] > 0 and arr[1] > 0:
        return 'Happy'
    elif arr[0] < 0 and arr[1] > 0:
        return 'Angry'
    elif arr[0] > 0 and arr[1] < 0:
        return 'Calm'
    elif arr[0] < 0 and arr[1] < 0:
        return 'Sad'
    else:
        return 'Neutral'

In [ ]:
# Your code here
sc = MinMaxScaler()
label_cols=['valence', 'arousal']
sc.fit_transform(data[label_cols].to_numpy(dtype=np.float32))
cap = cv2.VideoCapture(1)
print("Capture on")
emo_list=[]
flag=True
while flag:
    ret, frame = cap.read()

    if not ret:
        print("Error: Unable to capture the frame.")
        break

    detected_faces = detector.detect_faces(frame)
    detected_landmarks = detector.detect_landmarks(frame, detected_faces)
    detected_aus = detector.detect_aus(frame, detected_landmarks)
    for faces,au_units in zip(detected_faces,detected_aus): #access only one frame
        for i in range(len(faces)): #access all faces detected in the frame
            au_arr=[au_units[i][j] for j in [4,5,9,10,19] ]
            #print(au_arr)
            au_arrr=model(torch.tensor(au_arr).to(device))
            #print(sc.inverse_transform(np.array(au_arrr.cpu()).reshape(1, -1))[0])
            emotion=return_emo_state(sc.inverse_transform(np.array(au_arrr.cpu()).reshape(1, -1))[0])
            emo_list.append(emotion)
            x, y, w, h, p = faces[i]
            # Drawing a rectangle around the detected face
            cv2.rectangle(frame, (int(x), int(y)), (int(w), int(h)), (0,0 , 255), 2)

            # Displaying the emotion label on top of the rectangle
            cv2.putText(frame, emotion, (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Displaying the frame with detected faces and emotions
            jcv2.imshow("Emotion Detection", frame)

            # Press Esc to exit the program
            key = jcv2.waitKey(1) or 0xFF
            if key == 27:
                flag=False
                break
        
cap.release()
jcv2.destroyAllWindows()
print("Capture off")
#[[0.47976798 0.3999247 ]]

In [75]:
import pandas as pd

d={'emotion': emo_list}
d_df=pd.DataFrame(d)


In [76]:
d_df.value_counts()


emotion
Happy      38
Angry      16
Name: count, dtype: int64